# The battle of neighbourhoods Week 2

In this Week im goint to check the diferences between the neighbourhoods of the East Toronto Borough, trying to understand what kind of business are well rated in each neighbourhoods and helping a business man that would like to know where to open a new restaurant and what kind of restaurant should it open in East Toronto Borough

In order to do that, we are going to use the dataset from the previous week, filtering the final dataframe by East Toronto.
After that we are going to get some information about the shops in that area trying to find a market niche to deploy a new shop
with more odds of success

The following code lines are for showing how i achieved the main data set for this proposal

In [1]:
import numpy as np 
import pandas as pd
from bs4 import BeautifulSoup
import xml

import json
import requests 
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



Getting the wikipedi data and finding the scope fields by tagging the html
note that im using the old wikipedia ID

In [2]:
url = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.').text
soup = BeautifulSoup(url,'lxml')

In [3]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Deleting Not assigned Borough & Grouping Neighbourhoods and joining in a simple field separated by ,

In [4]:
df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.columns = ['Postcode', 'Borough', 'Neighbourhood']

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [5]:
df["Neighbourhood"]=np.where(df['Neighbourhood']=='Not assigned', df['Borough'],df['Neighbourhood'])

# Getting latitude & longitude

In [6]:
df_g = pd.read_csv('http://cocl.us/Geospatial_data')
df_g.columns = ['Postcode', 'Latitude', 'Longitude']

join the two dataframes by Postcode Field

In [7]:
df_p = pd.merge(df, df_g, on=['Postcode'], how='inner')

df_Toronto = df_p[['Borough', 'Neighbourhood', 'Postcode', 'Latitude', 'Longitude']].copy()

df_Toronto.head()

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


Selecting only East Toronto and West Toronto values

In [60]:
df_t = df_Toronto[(df_Toronto['Borough']=='East Toronto')] #| (df_Toronto['Borough']=='West Toronto')]

toronto = df_t.reset_index(drop=True)
toronto

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
2,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572
3,East Toronto,Studio District,M4M,43.659526,-79.340923
4,East Toronto,Business Reply Mail Processing Centre 969 Eastern,M7Y,43.662744,-79.321558


In [61]:
# Working with Foursquare
CLIENT_ID = 'HRQJOTB4DINAGDKAJ0OJ5E0FWFES3HKQRHTGKP2GGBPTVA0B' # your Foursquare ID
CLIENT_SECRET = 'O333C1WCXI14Q1QIAXHZR0M31PWTXVAKX020SP3M11P4GH0P' # your Foursquare Secret
VERSION = '20200328' # Foursquare API version


In [85]:
radius = 200
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Postcode'], toronto['Borough'], 
                                                  toronto['Neighbourhood']):
    #url = "https://api.foursquare.com/v2/venues/explore?client_id=ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ&client_secret=OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H&v=20180605 \
    # &ll=43.653963,-79.387207&radius=10&limit=100".format(
    #    CLIENT_ID,
    #    CLIENT_SECRET,
    #    VERSION,
    #    lat,
    #    long,
    #    radius, 
    #    LIMIT)
    url ='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,VERSION, lat, long, radius, LIMIT)

    results = requests.get(url).json()["response"]['groups']
    
    for group in results:
        for item in group['items']:
            print(item)
            venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            item['venue']['name'], 
            item['venue']['location']['lat'], 
            item['venue']['location']['lng'],  
            item['venue']['categories'][0]['name']))

{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4bd461bc77b29c74a07d9282', 'name': 'Glen Manor Ravine', 'location': {'address': 'Glen Manor', 'crossStreet': 'Queen St.', 'lat': 43.67682094413784, 'lng': -79.29394208780985, 'labeledLatLngs': [{'label': 'display', 'lat': 43.67682094413784, 'lng': -79.29394208780985}], 'distance': 89, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Glen Manor (Queen St.)', 'Toronto ON', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d159941735', 'name': 'Trail', 'pluralName': 'Trails', 'shortName': 'Trail', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4bd461bc77b29c74a07d9282-0'}
{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reaso

In [86]:
venues

[('M4E',
  'East Toronto',
  'The Beaches',
  43.67635739999999,
  -79.2930312,
  'Glen Manor Ravine',
  43.67682094413784,
  -79.29394208780985,
  'Trail'),
 ('M4E',
  'East Toronto',
  'The Beaches',
  43.67635739999999,
  -79.2930312,
  'Glen Stewart Ravine',
  43.67629984029563,
  -79.2947841389563,
  'Other Great Outdoors'),
 ('M4E',
  'East Toronto',
  'The Beaches',
  43.67635739999999,
  -79.2930312,
  'Glen Stewart Park',
  43.67527822698259,
  -79.2946465494184,
  'Park'),
 ('M4K',
  'East Toronto',
  'The Danforth West, Riverdale',
  43.6795571,
  -79.352188,
  'Kalyvia',
  43.6779727,
  -79.3512077,
  'Greek Restaurant'),
 ('M4L',
  'East Toronto',
  'The Beaches West, India Bazaar',
  43.6689985,
  -79.31557159999998,
  'British Style Fish & Chips',
  43.66872336908457,
  -79.31713935234417,
  'Fish & Chips Shop'),
 ('M4L',
  'East Toronto',
  'The Beaches West, India Bazaar',
  43.6689985,
  -79.31557159999998,
  'Orchard Park',
  43.668858311782124,
  -79.31416022446447,

In [87]:
venues_df = pd.DataFrame(venues)

venues_df.columns = ['PostalCode', 'Borough', 'Neighbourhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
venues_df

,PostalCode,Borough,Neighbourhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
3,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Kalyvia,43.677973,-79.351208,Greek Restaurant
4,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,British Style Fish & Chips,43.668723,-79.317139,Fish & Chips Shop
5,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,Orchard Park,43.668858,-79.314160,Park
6,M4M,East Toronto,Studio District,43.659526,-79.340923,Ed's Real Scoop,43.660656,-79.342019,Ice Cream Shop
7,M4M,East Toronto,Studio District,43.659526,-79.340923,Leslieville Pumps,43.660892,-79.340626,Sandwich Place
8,M4M,East Toronto,Studio District,43.659526,-79.340923,Queen Books,43.660651,-79.342267,Bookstore
9,M4M,East Toronto,Studio District,43.659526,-79.340923,The Bone House,43.660894,-79.341097,Pet Store


In [91]:
venues_df2=venues_df.groupby(['PostalCode','Borough','Neighbourhood','VenueCategory'])['VenueName'].count().reset_index()
venues_df2

,PostalCode,Borough,Neighbourhood,VenueCategory,VenueName
0,M4E,East Toronto,The Beaches,Other Great Outdoors,1
1,M4E,East Toronto,The Beaches,Park,1
2,M4E,East Toronto,The Beaches,Trail,1
3,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,1
4,M4L,East Toronto,"The Beaches West, India Bazaar",Fish & Chips Shop,1
5,M4L,East Toronto,"The Beaches West, India Bazaar",Park,1
6,M4M,East Toronto,Studio District,Bakery,1
7,M4M,East Toronto,Studio District,Bank,1
8,M4M,East Toronto,Studio District,Bar,1
9,M4M,East Toronto,Studio District,Bookstore,1


Analizing the Data we can see that there are two options to deploy a business:

    - Option 1: the more risky way its try to open a shop/restaurant in the M7Y, M4E, M4K, M4L neighbourhoods. Why? it seems
      clear that in this neighbourhoods for some reason there are not restaurants or coffes, that could be because its not a         residential area, furthermore, we can see that there isnt shops, so it isnt too a good commercial area (the people             usually go to shopping to a area with more shops in order to find the best option.
     
    - Option 2: the Sdudio District is the most popular neighbourhood in East Toronto, ther are shops, coffee shops, and much         more business, so it seems to be the most crowded area in the borough.
    
After the analysis we decided to open a new chill out restaurant/pub in order to feed the people that came to spend the time in this area, and in the nights we are open in order to give cocktails to the clients.